In [31]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine
import MySQLdb

# Export Game Data

In [2]:
# Extract CSVs into DataFrames
game = "game.csv"
game_df = pd.read_csv(game)

In [3]:
game_df.head()

,game_id,season,type,date_time,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2012030221,20122013,P,2013-05-16T23:30:00Z,3,6,2,3,home win OT,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT
1,2012030222,20122013,P,2013-05-19T19:00:00Z,3,6,2,5,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT
2,2012030223,20122013,P,2013-05-21T23:30:00Z,6,3,2,1,away win REG,right,Madison Square Garden,/api/v1/venues/null,America/New_York,-4,EDT
3,2012030224,20122013,P,2013-05-23T23:00:00Z,6,3,3,4,home win OT,right,Madison Square Garden,/api/v1/venues/null,America/New_York,-4,EDT
4,2012030225,20122013,P,2013-05-25T21:30:00Z,3,6,1,3,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT


In [4]:
team_info = "team_info.csv"
team_info_df = pd.read_csv(team_info)

# Export Team Info Data

In [5]:
team_info_df.head()

,team_id,franchiseId,shortName,teamName,abbreviation,link
0,1,23,New Jersey,Devils,NJD,/api/v1/teams/1
1,4,16,Philadelphia,Flyers,PHI,/api/v1/teams/4
2,26,14,Los Angeles,Kings,LAK,/api/v1/teams/26
3,14,31,Tampa Bay,Lightning,TBL,/api/v1/teams/14
4,6,6,Boston,Bruins,BOS,/api/v1/teams/6


# Transform Game DataFrame

In [6]:
game_df['type'] = game_df['type'][game_df['type'] == 'P']
game_df = game_df.dropna()
game_df = game_df[['game_id', 'season', 'away_team_id', 'home_team_id', 'away_goals', 'home_goals','outcome']]
game_df.head()

,game_id,season,away_team_id,home_team_id,away_goals,home_goals,outcome
0,2012030221,20122013,3,6,2,3,home win OT
1,2012030222,20122013,3,6,2,5,home win REG
2,2012030223,20122013,6,3,2,1,away win REG
3,2012030224,20122013,6,3,3,4,home win OT
4,2012030225,20122013,3,6,1,3,home win REG


In [7]:
game_df= game_df.rename(columns={'away_team_id':'team_id', 'home_team_id':'home_team_id'})

game_df.head()

,game_id,season,team_id,home_team_id,away_goals,home_goals,outcome
0,2012030221,20122013,3,6,2,3,home win OT
1,2012030222,20122013,3,6,2,5,home win REG
2,2012030223,20122013,6,3,2,1,away win REG
3,2012030224,20122013,6,3,3,4,home win OT
4,2012030225,20122013,3,6,1,3,home win REG


# Transform Team_Info DataFrame

In [8]:
team_info_df = team_info_df[['team_id', 'shortName', 'teamName']]
team_info_df

,team_id,shortName,teamName
0,1,New Jersey,Devils
1,4,Philadelphia,Flyers
2,26,Los Angeles,Kings
3,14,Tampa Bay,Lightning
4,6,Boston,Bruins
5,3,NY,Rangers
6,5,Pittsburgh,Penguins
7,17,Detroit,Red Wings
8,28,San Jose,Sharks
9,18,Nashville,Predators


In [9]:
team_info_df = team_info_df.rename(columns={"team_id": "team_id" , "shortName": "CityName", "teamName": "TeamName"})
team_info_df.head()

,team_id,CityName,TeamName
0,1,New Jersey,Devils
1,4,Philadelphia,Flyers
2,26,Los Angeles,Kings
3,14,Tampa Bay,Lightning
4,6,Boston,Bruins


In [10]:
team_info_df['team_name'] = team_info_df["CityName"].str.cat(team_info_df["TeamName"], sep=' ')
team_info_df = team_info_df[['team_id','team_name']]
team_info_df = team_info_df.sort_values('team_id').reset_index()
team_info_df = team_info_df[['team_id','team_name']]
team_info_df.head()

,team_id,team_name
0,1,New Jersey Devils
1,2,NY Islanders
2,3,NY Rangers
3,4,Philadelphia Flyers
4,5,Pittsburgh Penguins


# Transform Data Into One Table

In [11]:
#game_df team_info_df away_team home_team

In [12]:
game_df = game_df.merge(team_info_df, on='team_id')

In [13]:
game_df = game_df.rename(columns={'team_name':"away_team"})
game_df = game_df.drop(columns='team_id')
game_df

,game_id,season,home_team_id,away_goals,home_goals,outcome,away_team
0,2012030221,20122013,6,2,3,home win OT,NY Rangers
1,2012030222,20122013,6,2,5,home win REG,NY Rangers
2,2012030225,20122013,6,1,3,home win REG,NY Rangers
3,2012030131,20122013,15,1,3,home win REG,NY Rangers
4,2012030132,20122013,15,0,1,home win OT,NY Rangers
5,2012030135,20122013,15,1,2,home win OT,NY Rangers
6,2012030137,20122013,15,5,0,away win REG,NY Rangers
7,2015030141,20152016,5,2,5,home win REG,NY Rangers
8,2015030142,20152016,5,4,2,away win REG,NY Rangers
9,2015030145,20152016,5,3,6,home win REG,NY Rangers


In [14]:
game_df = game_df.rename(columns={'home_team_id':'team_id'})
game_df = game_df.merge(team_info_df, on='team_id')
game_df

,game_id,season,team_id,away_goals,home_goals,outcome,away_team,team_name
0,2012030221,20122013,6,2,3,home win OT,NY Rangers,Boston Bruins
1,2012030222,20122013,6,2,5,home win REG,NY Rangers,Boston Bruins
2,2012030225,20122013,6,1,3,home win REG,NY Rangers,Boston Bruins
3,2012030313,20122013,6,1,2,home win OT,Pittsburgh Penguins,Boston Bruins
4,2012030314,20122013,6,0,1,home win REG,Pittsburgh Penguins,Boston Bruins
5,2013030111,20132014,6,1,0,away win REG,Detroit Red Wings,Boston Bruins
6,2013030112,20132014,6,1,4,home win REG,Detroit Red Wings,Boston Bruins
7,2013030115,20132014,6,2,4,home win REG,Detroit Red Wings,Boston Bruins
8,2012030413,20122013,6,0,2,home win REG,Chicago Blackhawks,Boston Bruins
9,2012030414,20122013,6,6,5,away win OT,Chicago Blackhawks,Boston Bruins


In [15]:
game_df = game_df.rename(columns={'team_name':"home_team"})
game_df = game_df.drop(columns='team_id')
game_df

,game_id,season,away_goals,home_goals,outcome,away_team,home_team
0,2012030221,20122013,2,3,home win OT,NY Rangers,Boston Bruins
1,2012030222,20122013,2,5,home win REG,NY Rangers,Boston Bruins
2,2012030225,20122013,1,3,home win REG,NY Rangers,Boston Bruins
3,2012030313,20122013,1,2,home win OT,Pittsburgh Penguins,Boston Bruins
4,2012030314,20122013,0,1,home win REG,Pittsburgh Penguins,Boston Bruins
5,2013030111,20132014,1,0,away win REG,Detroit Red Wings,Boston Bruins
6,2013030112,20132014,1,4,home win REG,Detroit Red Wings,Boston Bruins
7,2013030115,20132014,2,4,home win REG,Detroit Red Wings,Boston Bruins
8,2012030413,20122013,0,2,home win REG,Chicago Blackhawks,Boston Bruins
9,2012030414,20122013,6,5,away win OT,Chicago Blackhawks,Boston Bruins


In [16]:
neworder = ['game_id','season', 'away_team','home_team','away_goals','home_goals','outcome']
playoff_data = game_df.reindex(columns=neworder)
playoff_data

,game_id,season,away_team,home_team,away_goals,home_goals,outcome
0,2012030221,20122013,NY Rangers,Boston Bruins,2,3,home win OT
1,2012030222,20122013,NY Rangers,Boston Bruins,2,5,home win REG
2,2012030225,20122013,NY Rangers,Boston Bruins,1,3,home win REG
3,2012030313,20122013,Pittsburgh Penguins,Boston Bruins,1,2,home win OT
4,2012030314,20122013,Pittsburgh Penguins,Boston Bruins,0,1,home win REG
5,2013030111,20132014,Detroit Red Wings,Boston Bruins,1,0,away win REG
6,2013030112,20132014,Detroit Red Wings,Boston Bruins,1,4,home win REG
7,2013030115,20132014,Detroit Red Wings,Boston Bruins,2,4,home win REG
8,2012030413,20122013,Chicago Blackhawks,Boston Bruins,0,2,home win REG
9,2012030414,20122013,Chicago Blackhawks,Boston Bruins,6,5,away win OT


# Load Data Into Database

In [32]:
# Create database connection
rds_connection_string = "root:Password1@localhost/playoff_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [33]:
# Confirm tables
engine.table_names()

['playoff_data']

In [34]:
# Load data into Database
playoff_data.to_sql(name='playoff_data', con=engine, if_exists='replace', index=False)

# Queries

In [36]:
pd.read_sql_query('select * from playoff_data', con=engine).head()

,game_id,season,away_team,home_team,away_goals,home_goals,outcome
0,2012030221,20122013,NY Rangers,Boston Bruins,2,3,home win OT
1,2012030222,20122013,NY Rangers,Boston Bruins,2,5,home win REG
2,2012030225,20122013,NY Rangers,Boston Bruins,1,3,home win REG
3,2012030313,20122013,Pittsburgh Penguins,Boston Bruins,1,2,home win OT
4,2012030314,20122013,Pittsburgh Penguins,Boston Bruins,0,1,home win REG
